In [ ]:
'''
Generate training data for analysis with Lanternfish

Lanternfish represents motion as a 3D image with dimensions
x, y, and time (t). These images are referred to as 'motion cubes'.
These cubes are saved as 3D numpy arrays in compressed .npz archives.

As input, you'll need X and Y coordinates for each object stored
in an N x T CSV, where N is the number of objects and T is the number of
time steps. One CSV stores X coordinates and another Y coordinates.
i.e.) tracksX(i, t) represents the object i's X coordinate at time t. 

This notebook outlines how to generate 3D motion cubes from simulated
random walk, power flier, and fractal Brownian motion paths.
'''
import numpy as np
from motion_cube import motion_cube, crop_tracks, gauss_kernel
from scipy import ndimage as ndi
from skimage.morphology import disk
from skimage.filters import gaussian
import os

In [ ]:
# set directory locations
save_dir = '/path/to/save/motioncubes/'
rw_dir = os.path.join(save_dir, 'rw_cubes')
pf_dir = os.path.join(save_dir, 'pf_cubes')
fbm_dir = os.path.join(save_dir, 'fbm_cubes')
# Set the compression factor
# Compression is necc. due to GPU memory restrictions
# Up to 4X compression doesn't appear to inhibit training
fold_compression = 2

In [ ]:
# Load paths
rw_x = np.loadtxt('sim_data/rw_X_100k.csv', delimiter=',')
rw_y = np.loadtxt('sim_data/rw_Y_100k.csv', delimiter=',')
pf_x = np.loadtxt('sim_data/pf_X_100k.csv', delimiter=',')
pf_y = np.loadtxt('sim_data/pf_Y_100k.csv', delimiter=',')
fbm_x = np.loadtxt('sim_data/fBm_X_100k.csv', delimiter=',')
fbm_y = np.loadtxt('sim_data/fBm_Y_100k.csv', delimiter=',')

In [ ]:
# Compress paths
rw_x = rw_x / fold_compression
rw_y = rw_y / fold_compression
pf_x = pf_x / fold_compression
pf_y = pf_y / fold_compression
# fBm sims travel farther from the origin, so compress them 2X as much
fbm_x = fbm_x / (fold_compression * 2)
fbm_y = fbm_y / (fold_compression * 2)

In [ ]:
# Crop outlier tracks that wander very far from the origin
# This allows for use of smaller motion cubes, preventing a few
# outlier cubes from 'diluting' the feature space of most cubes
# with a bunch of empty pixels
rw_x, rw_y = crop_tracks(rw_x, rw_y, lbound=64, hbound=78)
pf_x, pf_y = crop_tracks(pf_x, pf_y, lbound=64, hbound=78)
fbm_x, fbm_y = crop_tracks(fbm_x, fbm_y, lbound=64, hbound=78)

In [ ]:
# Perform class balancing
min_class = min(rw_x.shape[0], pf_x.shape[0], fbm_x.shape[0])
min_class = 15000
rw_x, rw_y = rw_x[:min_class, :], rw_y[:min_class,:]
pf_x, pf_y = pf_x[:min_class, :], pf_y[:min_class,:]
fbm_x, fbm_y = fbm_x[:min_class, :], fbm_y[:min_class,:]

In [ ]:
# Set cube size
x_max, y_max = 156, 156

In [ ]:
# Specify a position marker to use as a 2D ndarray. 
# Binary disks seem to work well as position markers, and broad
# Gaussians have also been effectacious.
width = 25
sigma = 20
staticK = np.zeros([x_max*2, y_max*2]) # kernel must be 2X size of cube
staticK[x_max,y_max] = 1
se = disk(width)
staticK = ndi.binary_dilation(staticK, structure=se)

In [ ]:
# Generate motion cubes
rw_cubes = motion_cube(rw_x, rw_y, x_max, y_max, staticK=staticK, binary=True, save=rw_dir)
pf_cubes = motion_cube(pf_x, pf_y, x_max, y_max, staticK=staticK, binary=True, save=pf_dir)
fbm_cubes = motion_cube(fbm_x, fbm_y, x_max, y_max, staticK=staticK, binary=True, save=fbm_dir)